# Advent code 2022 day 15 solution
### Mathew Garvin 11/01/2023

Assumptions
 - 

In [1]:
from urllib.request import urlopen
input_source = "https://raw.githubusercontent.com/MathewGarvin/Advent2022/main/advent2022day15input.txt"

def golf_score(code_string):
    return len(code_string.replace(' ','').replace('\n',''))

test_input = b"""Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
"""
print(test_input)

b'Sensor at x=2, y=18: closest beacon is at x=-2, y=15\nSensor at x=9, y=16: closest beacon is at x=10, y=16\nSensor at x=13, y=2: closest beacon is at x=15, y=3\nSensor at x=12, y=14: closest beacon is at x=10, y=16\nSensor at x=10, y=20: closest beacon is at x=10, y=16\nSensor at x=14, y=17: closest beacon is at x=10, y=16\nSensor at x=8, y=7: closest beacon is at x=2, y=10\nSensor at x=2, y=0: closest beacon is at x=2, y=10\nSensor at x=0, y=11: closest beacon is at x=2, y=10\nSensor at x=20, y=14: closest beacon is at x=25, y=17\nSensor at x=17, y=20: closest beacon is at x=21, y=22\nSensor at x=16, y=7: closest beacon is at x=15, y=3\nSensor at x=14, y=3: closest beacon is at x=15, y=3\nSensor at x=20, y=1: closest beacon is at x=15, y=3\n'


## Part 1 solution with tracing output

In [123]:
text_colour = lambda r, g, b: "\x1b[38;2;{};{};{}m".format(r, g, b)
no_text_colour = "\x1b[39m"
back_colour = lambda r, g, b: "\x1b[48;2;{};{};{}m".format(r, g, b)
no_back_colour = "\x1b[49m"

def visualise():
    pass

In [28]:
m_dist = lambda z: abs(z.real) + abs(z.imag)

def part1(y, debug=False):
    
    if debug:
        input_string = test_input
        
    else:
        f = urlopen(input_source)
        input_string = f.read()
        f.close()
        
    input_string = input_string[12:-1]
    a = input_string.split(b"\nSensor at x=")
    b = [c.split(b": closest beacon is at x=") for c in a]
    input_scan = [[complex(*[int(g) for g in f.split(b", y=")]) for f in e] for e in b]
    input_scan = [[*a, m_dist(a[0] - a[1])] for a in input_scan]
    
    if debug:
        print(input_scan)
        
    left_edge = int(min([a[0].real - m_dist(a[0] - a[1]) for a in input_scan]))
    right_edge = int(max([a[0].real + m_dist(a[0] - a[1]) for a in input_scan]))
    print(left_edge, right_edge)
    
    non_beacons = set()
    
    for a in range(left_edge, right_edge + 1):
        
        for b in input_scan:
            
            if debug:
                print(a, b[0], m_dist(b[0] - a - y), b[2], 
                      m_dist(b[0] - a - y) <= b[2] and a + y != b[1])
                
            if m_dist(b[0] - a - y) <= b[2] and a + y != b[1]:
                non_beacons.add(a + y)
            
    if debug:
        print(non_beacons)
        
    return len(non_beacons)
        
part1(2000000j, False)

-1469700 5315149


4811413

In [14]:
for a in range(1+1j, 5+1j, 1):
    print(a)

TypeError: 'complex' object cannot be interpreted as an integer

## Part 2 solution with tracing output

In [265]:
def visualise2(grid):
    sand_col = [224, 200, 92]
    rock_col = [92, 92, 92]
    text_col = sand_col
    vis = text_colour(*text_col)
    
    for line in grid:
        
        for char in line:
            
            if char == 0:
                vis += "."
                
            if char == 1:
                
                if text_col != rock_col:
                    text_col = rock_col
                    vis += text_colour(*text_col)
                    
                vis += "#"
                
            if char == 2:
                
                if text_col != sand_col:
                    text_col = sand_col
                    vis += text_colour(*text_col)
                    
                vis += "o"
                
        vis += "\n"
        
    print(no_text_colour + vis)

In [267]:
def part2(floor=True, debug=False):
    
    if debug:
        input_string = test_input
        
    else:
        f = urlopen(input_source)
        input_string = f.read()[:-1]
        f.close()

    input_scan = [[complex(*[int(c) for c in b.split(b",")]) for b in a.split(b" -> ")] for a in input_string.split(b"\n")]
    
    if debug:
        print(input_scan)

    blocks = []

    for line in input_scan:

        for start_block, end_block in zip(line[:-1], line[1:]):
            
            if debug:
                print(start_block, end_block, end_block - start_block, abs(end_block - start_block))

            for n in range(1 + int(abs(end_block - start_block))):
                b = start_block + n * (end_block - start_block) / abs(end_block - start_block)
                blocks.append(b)

    origin = 500
    lower_edge = int(2 + max([b.imag for b in blocks]))
    left_edge = int(min(min([b.real for b in blocks]), origin - lower_edge))
    right_edge = int(max(max([b.real for b in blocks]), origin + lower_edge))
    
    grid = [[1 - (a * floor < lower_edge)] * (right_edge - left_edge + 1) for a in range(lower_edge + 1)]
    
    for b in blocks:
        grid[int(b.imag)][int(b.real) - left_edge] = 1
        
    moving = True

    while moving and grid[0][origin - left_edge] == 0:        
        old_grain = 0
        new_grain = origin

        while old_grain != new_grain:
            old_grain = new_grain

            if new_grain.imag >= lower_edge:
                moving = False
                
            elif grid[int(new_grain.imag + 1)][int(new_grain.real) - left_edge] < 1:
                new_grain += 1j

            elif grid[int(new_grain.imag) + 1][int(new_grain.real) - left_edge - 1] < 1:
                new_grain -= 1 - 1j
                
            elif grid[int(new_grain.imag) + 1][int(new_grain.real) - left_edge + 1] < 1:
                new_grain += 1 + 1j
                
        grid[int(new_grain.imag)][int(new_grain.real) - left_edge] = 2 * moving

    if debug:
        print(grid)
        
    visualise2(grid)
    print(sum(a.count(2) for a in grid))
    
part2(True, False)

........................................................................................................................................................................o........................................................................................................................................................................
.......................................................................................................................................................................ooo.......................................................................................................................................................................
......................................................................................................................................................................ooooo.........................................................................................................................................................

## Parts 1 & 2 combined solution

In [341]:
f = urlopen(input_source)
#------------------------

I = int
R = range
    
def F(p):
    k = []

    for n in [[C(*[I(c) for c in b.split(b",")]) 
               for b in a.split(b" -> ")] for a in s.split(b"\n")]:
        
        for a, z in zip(n[:-1], n[1:]):
            d = z - a

            for n in R(1 + I(abs(d))):
                k += [a + n * d / abs(d)]

    o = 500
    d = I(max([b.imag for b in k])) + 2
    l = I(min(*[b.real for b in k], o - d))
    r = I(max(*[b.real for b in k], o + d)) - l
    o -= l
    
    g = [[1 - (a * p < d)] * (r + 1) for a in R(d + 1)]
    
    for b in k:
        g[I(b.imag)][I(b.real) - l] = 1
        
    m = 2

    while m and g[0][o] == 0:        
        p = 0
        q = o

        while p != q:
            p = q
            r = I(q.real)
            i = I(q.imag)

            if i >= d:
                m = 0
                
            elif g[i + 1][r] < 1:
                q += 1j

            elif g[i + 1][r - 1] < 1:
                q -= 1 - 1j
                
            elif g[i + 1][r + 1] < 1:
                q += 1 + 1j
                
        g[i][r] = m
        
    print(sum(a.count(2) for a in g))

    
s = f.read()[:-1]
F(0)   
F(1)

#------------------------
f.close()

NameError: name 'bytestring' is not defined

In [304]:
class C(complex):
    R = lambda s: int(s.real)
    I = lambda s: int(s.imag)

a = C(3, 7)
a.R()
print(type(a))

<class '__main__.C'>


Check substantive code length for golfing score

In [344]:
code = """
I = int
R = range
    
def F(p):
    k = []

    for n in [[C(*[I(c) for c in b.split(b",")]) 
               for b in a.split(b" -> ")] for a in s.split(b"\n")]:
        
        for a, z in zip(n[:-1], n[1:]):
            d = z - a

            for n in R(1 + I(abs(d))):
                k += [a + n * d / abs(d)]

    o = 500
    d = I(max([b.imag for b in k])) + 2
    l = I(min(*[b.real for b in k], o - d))
    r = I(max(*[b.real for b in k], o + d)) - l
    o -= l
    
    g = [[1 - (a * p < d)] * (r + 1) for a in R(d + 1)]
    
    for b in k:
        g[I(b.imag)][I(b.real) - l] = 1
        
    m = 2

    while m and g[0][o] == 0:        
        p = 0
        q = o

        while p != q:
            p = q
            r = I(q.real)
            i = I(q.imag)

            if i >= d:
                m = 0
                
            elif g[i + 1][r] < 1:
                q += 1j

            elif g[i + 1][r - 1] < 1:
                q -= 1 - 1j
                
            elif g[i + 1][r + 1] < 1:
                q += 1 + 1j
                
        g[i][r] = m
        
    print(sum(a.count(2) for a in g))

    
s = f.read()[:-1]
F(0)   
F(1)
"""
golf_score(code)

543

In [131]:
golf_score("(S([P(i, [[2]]) for i in x]) + 2 + L(x))")

31